In [9]:
import yaml
import os
from glob import glob
import os
import pandas as pd
import json
import pickle
import numpy as np
import wandb
import torch
import sys 

sys.path.insert(1,'/home/lmeyers/ReID_complete/')

from pytorch_data import *
from pytorch_models import *
from pytorch_train_and_eval_color_detect import *

In [10]:
config_file = '/home/lmeyers/ReID_complete/color_detect_experiments/8_ids_double_colors_batch1_sample_num_64/8_ids_double_colors_batch1_sample_num_64.yml' #'/home/lmeyers/ReID_complete/color_detect_experiments/_ids_batch1_sample_num_08/_ids_batch1_sample_num_08.yml'

with open(config_file, 'r') as fo:
    config = yaml.safe_load(fo)
model_config = config['model_settings'] # settings for model building
train_config = config['train_settings'] # settings for model training
data_config = config['data_settings'] # settings for data loading
eval_config = config['eval_settings'] # settings for evaluation
torch_seed = config['torch_seed']
verbose = config['verbose']

In [15]:
run_to_restart =  '9xz07aa4' #'k4il16yj'
check_point_to_use = 200


train_config['wandb_run_id'] = run_to_restart
train_config['checkpoint_to_load'] = check_point_to_use

#Load model
most_recent_epoch =  train_config['checkpoint_to_load']
print(f'Resuming training from saved epoch: {most_recent_epoch}')
most_recent_model = os.path.dirname(model_config['model_path'])+r'/checkpoints/'+str(most_recent_epoch)+'.pth'
print(f'Loading saved checkpoint model {most_recent_model}')
model = torch.load(most_recent_model)

#resart logging
#experiment = wandb.init(project= train_config["wandb_project_name"],entity=train_config['wandb_entity_name'],resume=True,id=train_config['wandb_run_id'],dir=train_config['wandb_dir_path'])

Resuming training from saved epoch: 200
Loading saved checkpoint model ./8_ids_double_colors_batch1_sample_num_64/checkpoints/200.pth


In [16]:
#SET GPU TO USE
os.environ["CUDA_VISIBLE_DEVICES"]=str(train_config['gpu'])
if verbose:
    print('Using GPU',train_config['gpu'])


# LOAD TO DEVICE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
if verbose:
    print(f'Device: {device}')
model.to(device)


Using GPU 1
Device: cuda


Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

In [17]:
train_file = data_config['datafiles']['train']
#train_file = root+r'/'+files[i]
wandb_name = 'color_detect_'+ os.path.basename(os.path.dirname(train_file))
run_str = os.path.basename(train_file)[36:-4]


split_parts = run_str.rsplit('_', 1)
# Check if there is at least one underscore in the string
if len(split_parts) > 1:
    # Get the substring after the last underscore
    num_images = split_parts[1]
    num_ids = split_parts[-1]
else:
    # Handle the case where there are no underscores in the string
    num_images = run_str

reference_file = config['data_settings']['datafiles']['reference']


test_file = config['data_settings']['datafiles']['test']


In [18]:

# evaluate on test set
if verbose:
    print('Evaluating model...')

## load test dataset 
test_fname = data_config['datafiles']['test']
df_test = pd.read_csv(test_fname)

dft_test = prepare_for_triplet_loss(df_test, data_config['label_col'], data_config['fname_col'])

# BUILD DATASET AND DATALOADER
test_dataset = ColorMap_w_Order(dft_test, 'filename', 'label',data_config['input_size'],'test',data_config['datafiles']['color_map'])
bs=64
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=False)
batch = next(iter(test_dataloader))

# PERFORM INFERENCE
with torch.no_grad():
    acc = []
    prec = []
    rec = []
    vals = {}
    corr = []
    for k, data in enumerate(test_dataloader):
        if verbose: 
            print('Performing Inference on Batch',k)
        images = data['image'].to(device)
        # get labels
        labels = data['label'].to(device)
        outputs = model(images)
        #matches = (torch.round(outputs) == labels)
        #print(matches.shape)
        #correct = np.all(matches,axis=1,out=None)
        #print(correct.shape)
        #matches = torch.logical_and(torch.round(outputs),labels)
        #predictions = torch.round(outputs) #larger than 0
        predictions = torch.where(outputs > 0,1, 0)
        mapping_array = np.zeros_like(predictions.cpu())  # Initialize with zero TN
        # Set elements to 1 where both binary arrays have value 1
        labels_cpu = labels.cpu()
        #predictions = 
        mapping_array[(predictions.cpu() == 1) & (labels_cpu == 1)] = 1 #TP
        mapping_array[(predictions.cpu() == 1) & (labels_cpu == 0)] = 2 #FP
        mapping_array[(predictions.cpu() == 0) & (labels_cpu == 1)] = 3 #FN
        #accuracy = (matches.float().mean())
        accuracy, precision, recall = getMetrics(outputs,labels)
        acc.append(accuracy)
        prec.append(precision)
        rec.append(recall)
        corrects = [2 and 3 not in sample for sample in mapping_array] #in is expensive (replace!)
        corr = np.concatenate((corr,corrects))
        for i in range(len(labels[0])):
            values_at_index = [subarray[i] for subarray in mapping_array]
            #print('Batch',k,'Class',i,values_at_index)
            if k == 0:
                vals[i] = values_at_index  
            else:
                vals[i] = np.concatenate((vals[i],values_at_index))
    class_dict = {}
    for c in vals:
            TP = torch.sum(torch.tensor(vals[c]) == 1)
            TN = torch.sum(torch.tensor(vals[c]) == 0)
            FP = torch.sum(torch.tensor(vals[c]) == 2)
            FN = torch.sum(torch.tensor(vals[c]) == 3)
            All = torch.tensor(len(vals[c]))
            class_acc = (TP + TN)/ All
            class_prec=  (TP)/(TP+FP)
            class_rec = (TP)/(TP+FN)
            class_dict[c]={'acc:':torch.tensor(class_acc).float().mean(),'precision:':torch.tensor(class_prec).float().mean(),'recall:':torch.tensor(class_rec).float().mean()}
            print('Class',c,'TP:',int(TP),'FN:',int(FN),'FP:',int(FP),'TN:',int(TN),'Acc:',torch.tensor(class_acc).float().mean(),'Precision:',torch.tensor(class_prec).float().mean(),'Recall:',torch.tensor(class_rec).float().mean(),)
    print('Total Correct Samples',np.sum(corr),"out of",len(corr),'=',(np.sum(corr)/len(corr)))
    print("Total Acc:",torch.tensor(acc).mean())
    print("Total Precision:",torch.tensor(prec).mean())
    print("Total Recall:",torch.tensor(rec).mean())

results = {'percent_correct_samples':(np.sum(corr)/len(corr)),"Total Acc:":float(torch.tensor(acc).mean()),"Total Precision:":float(torch.tensor(prec).mean()),"Total Recall:":float(torch.tensor(rec).mean()),"Class_dict":class_dict}

# Add total training loss to results 
results['train_loss'] = 'Null'
print(results)

# Adding other metrics to results to pass to csv
results['valid_loss'] = 'Null'
results['wandb_id'] = experiment.id
print(experiment.id)
results['start_time'] = experiment.start_time
results['train_time'] = 'Null'
results['stop_epoch'] = check_point_to_use


# Save results to temporary file
with open('/home/lmeyers/ReID_complete/color_detect_experiments/results.pkl','wb') as fi:
    pickle.dump(results,fi)

if model_config['model_path'] is not None:
    print('Saving model...')
    torch.save(model, model_config['model_path'])
else:
    print('model_path not provided. Not saving model')
print('Finished')
wandb.finish()


# Save model to wandb file location to prevent overwriting
#!cp {config['model_settings']['model_path']} {config['train_settings']['wandb_dir_path']+'/wandb/latest-run/files/'+os.path.basename(config['model_settings']['model_path'])}

with open('/home/lmeyers/ReID_complete/color_detect_experiments/results.pkl','rb') as fi:
    results = pickle.load(fi)

# Write out run summary to results tracking document
results_df = pd.read_csv(config['eval_settings']['results_file'])
results_df.loc[len(results_df)] = {'run_str': run_str,
                                    'wandb_id':results['wandb_id'],
                                    'num_ids':num_ids,
                                    'num_images_per_id':num_images,
                                    'total_training_images':len(pd.read_csv(train_file)),
                                    'batch_size':config['data_settings']['batch_size'],
                                    'num_epochs':config['train_settings']['num_epochs'],
                                    'train_loss':results['train_loss'],
                                    'valid_loss':results['valid_loss'],
                                    'percent_correct_samples':results['percent_correct_samples'],
                                    'total_acc':results["Total Acc:"],
                                    'total_precision':results["Total Precision:"],
                                    'total_recall':results["Total Recall:"],
                                    'training_file':train_file,
                                    'reference_file':reference_file,
                                    'query_file':test_file,
                                    'start_time':results['start_time'],
                                    'train_time':results['train_time'],
                                    'stop_epoch':results['stop_epoch']}
results_df.to_csv(config['eval_settings']['results_file'],index=False)




Evaluating model...


Performing Inference on Batch 0
Performing Inference on Batch 1
Performing Inference on Batch 2
Performing Inference on Batch 3
Performing Inference on Batch 4
Performing Inference on Batch 5
Performing Inference on Batch 6
Performing Inference on Batch 7
Performing Inference on Batch 8
Performing Inference on Batch 9
Performing Inference on Batch 10
Performing Inference on Batch 11
Performing Inference on Batch 12
Performing Inference on Batch 13
Performing Inference on Batch 14
Performing Inference on Batch 15
Performing Inference on Batch 16
Performing Inference on Batch 17
Performing Inference on Batch 18
Performing Inference on Batch 19
Performing Inference on Batch 20
Performing Inference on Batch 21
Performing Inference on Batch 22
Performing Inference on Batch 23
Performing Inference on Batch 24
Performing Inference on Batch 25
Performing Inference on Batch 26
Performing Inference on Batch 27
Performing Inference on Batch 28
Performing Inference on Batch 29
Performing Inference

/tmp/ipykernel_31140/1156764684.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  class_dict[c]={'acc:':torch.tensor(class_acc).float().mean(),'precision:':torch.tensor(class_prec).float().mean(),'recall:':torch.tensor(class_rec).float().mean()}
/tmp/ipykernel_31140/1156764684.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print('Class',c,'TP:',int(TP),'FN:',int(FN),'FP:',int(FP),'TN:',int(TN),'Acc:',torch.tensor(class_acc).float().mean(),'Precision:',torch.tensor(class_prec).float().mean(),'Recall:',torch.tensor(class_rec).float().mean(),)


In [8]:
with open('/home/lmeyers/ReID_complete/color_detect_experiments/results.pkl','rb') as fi:
    results = pickle.load(fi)

# Write out run summary to results tracking document
results_df = pd.read_csv(config['eval_settings']['results_file'])
results_df.loc[len(results_df)] = {'run_str': run_str,
                                    'wandb_id':results['wandb_id'],
                                    'num_ids':num_ids,
                                    'num_images_per_id':num_images,
                                    'total_training_images':len(pd.read_csv(train_file)),
                                    'batch_size':config['data_settings']['batch_size'],
                                    'num_epochs':config['train_settings']['num_epochs'],
                                    'train_loss':results['train_loss'],
                                    'valid_loss':results['valid_loss'],
                                    'percent_correct_samples':results['percent_correct_samples'],
                                    'total_acc':results["Total Acc:"],
                                    'total_precision':results["Total Precision:"],
                                    'total_recall':results["Total Recall:"],
                                    'training_file':train_file,
                                    'reference_file':reference_file,
                                    'query_file':test_file,
                                    'start_time':results['start_time'],
                                    'train_time':results['train_time'],
                                    'stop_epoch':results['stop_epoch']}
results_df.to_csv(config['eval_settings']['results_file'],index=False)